# SQL Injection Detection - ML Training Pipeline

**Project:** Cyber Security Intelligent Threat Mitigation  
**Stage:** ML Training & Model Comparison  
**Date:** November 2025

## Overview
This notebook trains and compares multiple ML models for SQL injection detection:
- **Baseline:** Logistic Regression
- **Main Models:** Random Forest & XGBoost
- **Features:** TF-IDF (char n-grams) + engineered numeric features

## Table of Contents
1. Environment Setup & Dependencies
2. Data Loading & Merging
3. Exploratory Data Analysis
4. Data Preprocessing & Cleaning
5. Feature Engineering
6. Model Training & Cross-Validation
7. Model Evaluation & Comparison
8. Explainability (SHAP, Feature Importance)
9. Threshold Tuning
10. Model Export & Integration

---

## 1. Environment Setup & Dependencies

Install required packages. If running on Google Colab, this cell will detect and mount Google Drive.

In [ ]:
# Detect if running on Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("✓ Running on Google Colab")
    
    # Mount Google Drive (optional)
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Install dependencies
    !pip install -q scikit-learn xgboost pandas numpy pyarrow shap joblib matplotlib seaborn imbalanced-learn
    
except ImportError:
    IN_COLAB = False
    print("✓ Running locally")

# Set random seeds for reproducibility
import random
import numpy as np

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

print(f"✓ Random seed set to {RANDOM_SEED}")

## 2. Import Libraries

In [ ]:
# Standard libraries
import os
import sys
import json
import time
import hashlib
import warnings
from pathlib import Path
from datetime import datetime
from collections import Counter

# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning - XGBoost only
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, 
    precision_recall_curve, roc_curve, auc,
    precision_score, recall_score, f1_score
)
from xgboost import XGBClassifier

# Model persistence
import joblib

# Suppress warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ All libraries imported successfully")
print(f"  - pandas: {pd.__version__}")
print(f"  - numpy: {np.__version__}")
print(f"  - scikit-learn: {__import__('sklearn').__version__}")
print(f"  - xgboost: {__import__('xgboost').__version__}")
print("\n✓ Note: This notebook uses XGBoost only for production deployment")

## 3. Configuration & Paths

Define all paths and hyperparameters

In [ ]:
# Project paths
if IN_COLAB:
    # Adjust these paths for Colab (upload data to Drive)
    PROJECT_ROOT = Path('/content/drive/MyDrive/cyber-poc')
else:
    PROJECT_ROOT = Path('../..')  # Assumes notebook is in ml/notebooks/

DATA_DIR = PROJECT_ROOT / 'data'
ML_DATA_DIR = PROJECT_ROOT / 'ml' / 'data'
ML_MODELS_DIR = PROJECT_ROOT / 'ml' / 'models'
ML_REPORTS_DIR = PROJECT_ROOT / 'ml' / 'reports'
DELIVERABLES_DIR = PROJECT_ROOT / 'deliverables'

# Create directories if they don't exist
for dir_path in [ML_DATA_DIR, ML_MODELS_DIR, ML_REPORTS_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

# Hyperparameters
CONFIG = {
    'random_seed': RANDOM_SEED,
    'test_size': 0.1,
    'val_size': 0.1,
    'max_duplicates': 3,
    
    # TF-IDF parameters
    'tfidf': {
        'max_features': 5000,
        'analyzer': 'char_wb',
        'ngram_range': (3, 6),
        'min_df': 2,
        'max_df': 0.95
    },
    
    # Random Forest parameters
    'rf_grid': {
        'n_estimators': [100, 200, 400],
        'max_depth': [None, 20, 40],
        'max_features': ['sqrt', 'log2'],
        'class_weight': ['balanced']
    },
    
    # XGBoost parameters
    'xgb_grid': {
        'n_estimators': [100, 300],
        'learning_rate': [0.05, 0.1],
        'max_depth': [6, 10],
        'subsample': [0.7, 1.0],
        'tree_method': ['hist']
    },
    
    # Cross-validation
    'cv_folds': 5,
    
    # Primary metric for model selection
    'primary_metric': 'f1_malicious'
}

print("✓ Configuration loaded")
print(f"  - Project root: {PROJECT_ROOT}")
print(f"  - ML data directory: {ML_DATA_DIR}")
print(f"  - TF-IDF max features: {CONFIG['tfidf']['max_features']}")

## 4. Data Loading & Merging

Load all available datasets and merge them into a single canonical dataset.

In [ ]:
def load_and_merge_datasets():
    """Load all available datasets and merge into canonical format"""
    
    datasets = []
    
    # 1. Load data/dataset.csv
    dataset_path = DATA_DIR / 'dataset.csv'
    if dataset_path.exists():
        print(f"Loading {dataset_path}...")
        df1 = pd.read_csv(dataset_path)
        df1['source_file'] = 'dataset.csv'
        datasets.append(df1)
        print(f"  ✓ Loaded {len(df1):,} rows")
    
    # 2. Load FINAL_DATASET_FOR_AI_TEAM_v3 (1).csv
    final_dataset_path = PROJECT_ROOT / 'FINAL_DATASET_FOR_AI_TEAM_v3 (1).csv'
    if final_dataset_path.exists():
        print(f"\nLoading {final_dataset_path.name}...")
        df2 = pd.read_csv(final_dataset_path)
        df2['source_file'] = 'FINAL_DATASET_FOR_AI_TEAM_v3.csv'
        datasets.append(df2)
        print(f"  ✓ Loaded {len(df2):,} rows")
    
    if not datasets:
        raise FileNotFoundError("No datasets found! Please ensure data files exist.")
    
    # Merge all datasets
    print(f"\nMerging {len(datasets)} dataset(s)...")
    df = pd.concat(datasets, ignore_index=True)
    
    print(f"✓ Total rows after merge: {len(df):,}")
    print(f"✓ Columns: {list(df.columns)}")
    
    return df

# Load data
df_raw = load_and_merge_datasets()

# Display sample
print("\nFirst 3 rows:")
df_raw.head(3)

## 5. Exploratory Data Analysis

In [ ]:
def analyze_dataset(df):
    """Perform EDA on the dataset"""
    
    print("=" * 80)
    print("DATASET ANALYSIS")
    print("=" * 80)
    
    # Basic info
    print(f"\n📊 Shape: {df.shape}")
    print(f"📊 Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Missing values
    print("\n📊 Missing values:")
    missing = df.isnull().sum()
    missing = missing[missing > 0].sort_values(ascending=False)
    if len(missing) > 0:
        for col, count in missing.items():
            print(f"  - {col}: {count:,} ({count/len(df)*100:.2f}%)")
    else:
        print("  ✓ No missing values")
    
    # Label distribution
    if 'is_malicious' in df.columns:
        print("\n📊 Label distribution:")
        label_counts = df['is_malicious'].value_counts()
        for label, count in label_counts.items():
            pct = count / len(df) * 100
            label_name = 'Malicious' if label == 1 else 'Benign'
            print(f"  - {label_name}: {count:,} ({pct:.2f}%)")
        
        # Class imbalance ratio
        if len(label_counts) == 2:
            imbalance_ratio = label_counts.max() / label_counts.min()
            print(f"  - Imbalance ratio: {imbalance_ratio:.2f}:1")
    
    # Attack types
    if 'attack_type' in df.columns:
        print("\n📊 Attack type distribution:")
        attack_counts = df['attack_type'].value_counts().head(10)
        for attack, count in attack_counts.items():
            if pd.notna(attack):
                print(f"  - {attack}: {count:,}")
    
    # Source files
    if 'source_file' in df.columns:
        print("\n📊 Source file distribution:")
        for source, count in df['source_file'].value_counts().items():
            print(f"  - {source}: {count:,}")
    
    # Query length statistics
    if 'raw_query' in df.columns:
        df['query_length'] = df['raw_query'].fillna('').str.len()
        print("\n📊 Query length statistics:")
        print(f"  - Min: {df['query_length'].min():.0f}")
        print(f"  - Max: {df['query_length'].max():.0f}")
        print(f"  - Mean: {df['query_length'].mean():.2f}")
        print(f"  - Median: {df['query_length'].median():.0f}")
    
    return df

# Analyze
df_raw = analyze_dataset(df_raw)

## 6. Visualization: Label Distribution

In [ ]:
# Plot label distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar plot
if 'is_malicious' in df_raw.columns:
    label_counts = df_raw['is_malicious'].value_counts()
    axes[0].bar(['Benign', 'Malicious'], [label_counts.get(0, 0), label_counts.get(1, 0)], 
                color=['green', 'red'], alpha=0.7)
    axes[0].set_ylabel('Count')
    axes[0].set_title('Label Distribution')
    axes[0].grid(axis='y', alpha=0.3)
    
    # Add counts on bars
    for i, v in enumerate([label_counts.get(0, 0), label_counts.get(1, 0)]):
        axes[0].text(i, v + len(df_raw)*0.01, f'{v:,}', ha='center', va='bottom')

# Pie chart
if 'is_malicious' in df_raw.columns:
    axes[1].pie([label_counts.get(0, 0), label_counts.get(1, 0)], 
                labels=['Benign', 'Malicious'],
                colors=['green', 'red'],
                autopct='%1.1f%%',
                startangle=90)
    axes[1].set_title('Label Proportion')

plt.tight_layout()
plt.savefig(ML_REPORTS_DIR / 'label_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Visualizations saved to ml/reports/")

## 7. Data Preprocessing & Cleaning

Clean and prepare data for training:
1. Remove rows with empty raw_query
2. Handle duplicates
3. Normalize schema
4. Remove PII (anonymize IPs)

In [ ]:
def preprocess_data(df, max_duplicates=3):
    """Clean and preprocess the dataset"""
    
    print("=" * 80)
    print("DATA PREPROCESSING")
    print("=" * 80)
    
    initial_rows = len(df)
    print(f"\nInitial rows: {initial_rows:,}")
    
    # 1. Remove rows with empty raw_query
    df = df[df['raw_query'].notna()]
    df = df[df['raw_query'].str.strip() != '']
    print(f"After removing empty queries: {len(df):,} ({initial_rows - len(df):,} removed)")
    
    # 2. Create query hash for deduplication
    df['query_hash'] = df['raw_query'].apply(
        lambda x: hashlib.md5(str(x).encode()).hexdigest()
    )
    
    # 3. Handle duplicates - keep max N duplicates
    before_dedup = len(df)
    duplicate_counts = df['query_hash'].value_counts()
    
    # For queries that appear > max_duplicates times, sample randomly
    keep_indices = []
    for query_hash in df['query_hash'].unique():
        query_indices = df[df['query_hash'] == query_hash].index
        if len(query_indices) <= max_duplicates:
            keep_indices.extend(query_indices)
        else:
            # Sample max_duplicates randomly
            sampled = np.random.choice(query_indices, size=max_duplicates, replace=False)
            keep_indices.extend(sampled)
    
    df = df.loc[keep_indices]
    print(f"After deduplication (max {max_duplicates} per query): {len(df):,} ({before_dedup - len(df):,} removed)")
    
    # 4. Anonymize IPs (optional - keep first 2 octets)
    if 'source_ip' in df.columns:
        df['source_ip'] = df['source_ip'].apply(
            lambda x: '.'.join(str(x).split('.')[:2]) + '.XXX.XXX' if pd.notna(x) else 'unknown'
        )
    
    # 5. Ensure label column
    if 'is_malicious' not in df.columns and 'label' in df.columns:
        df['is_malicious'] = df['label']
    
    # 6. Fill missing attack_type
    if 'attack_type' in df.columns:
        df['attack_type'] = df['attack_type'].fillna('benign')
    
    print(f"\n✓ Final dataset size: {len(df):,} rows")
    print(f"✓ Removed {initial_rows - len(df):,} rows total ({(initial_rows - len(df))/initial_rows*100:.2f}%)")
    
    return df

# Preprocess
df_clean = preprocess_data(df_raw, max_duplicates=CONFIG['max_duplicates'])

# Save cleaned dataset
df_clean.to_csv(ML_DATA_DIR / 'merged.csv', index=False)
df_clean.to_json(ML_DATA_DIR / 'merged.jl', orient='records', lines=True)

print(f"\n✓ Cleaned dataset saved to:")
print(f"  - {ML_DATA_DIR / 'merged.csv'}")
print(f"  - {ML_DATA_DIR / 'merged.jl'}")

## 8. Train/Val/Test Split

Split data into train (80%), validation (10%), and test (10%) sets with stratification.

In [ ]:
def create_splits(df, test_size=0.1, val_size=0.1, random_state=42):
    """Create stratified train/val/test splits"""
    
    print("=" * 80)
    print("CREATING DATA SPLITS")
    print("=" * 80)
    
    # First split: train+val vs test
    X = df['raw_query']
    y = df['is_malicious']
    
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=test_size, stratify=y, random_state=random_state
    )
    
    # Second split: train vs val
    val_size_adjusted = val_size / (1 - test_size)  # Adjust for already removed test set
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=val_size_adjusted, stratify=y_temp, random_state=random_state
    )
    
    print(f"\n📊 Split sizes:")
    print(f"  - Train: {len(X_train):,} ({len(X_train)/len(df)*100:.1f}%)")
    print(f"  - Val:   {len(X_val):,} ({len(X_val)/len(df)*100:.1f}%)")
    print(f"  - Test:  {len(X_test):,} ({len(X_test)/len(df)*100:.1f}%)")
    
    print(f"\n📊 Label distribution in splits:")
    for name, y_split in [('Train', y_train), ('Val', y_val), ('Test', y_test)]:
        counts = y_split.value_counts()
        benign = counts.get(0, 0)
        malicious = counts.get(1, 0)
        print(f"  {name:5s}: Benign={benign:,} ({benign/len(y_split)*100:.1f}%), "
              f"Malicious={malicious:,} ({malicious/len(y_split)*100:.1f}%)")
    
    return X_train, X_val, X_test, y_train, y_val, y_test

# Create splits
X_train, X_val, X_test, y_train, y_val, y_test = create_splits(
    df_clean, 
    test_size=CONFIG['test_size'],
    val_size=CONFIG['val_size'],
    random_state=CONFIG['random_seed']
)

print("\n✓ Data splits created")

## 9. Feature Engineering

Extract two types of features:
1. **TF-IDF features** - Character-level n-grams (3-6)
2. **Numeric features** - Hand-engineered counts and flags

In [ ]:
def extract_numeric_features(queries):
    """Extract hand-engineered numeric features from queries"""
    
    features = pd.DataFrame()
    
    # SQL keywords to check
    sql_keywords = ['UNION', 'SELECT', 'INSERT', 'UPDATE', 'DELETE', 'DROP', 
                    'CREATE', 'ALTER', 'EXEC', 'EXECUTE', 'SLEEP', 'WAITFOR',
                    'BENCHMARK', 'INFORMATION_SCHEMA', 'XP_CMDSHELL', 'LOAD_FILE']
    
    # Suspicious characters
    susp_chars = ["'", '"', '--', ';', '/*', '*/', '#']
    
    for query in queries:
        query_str = str(query).upper()
        
        feature_dict = {
            'len_raw': len(query_str),
            'count_single_quote': query_str.count("'"),
            'count_double_quote': query_str.count('"'),
            'count_dashes': query_str.count('--'),
            'count_semicolon': query_str.count(';'),
            'count_comment': query_str.count('/*') + query_str.count('*/')
 + query_str.count('#'),
            'count_susp_chars': sum(query_str.count(c) for c in susp_chars),
            'num_sql_keywords': sum(1 for kw in sql_keywords if kw in query_str),
            'has_union': int('UNION' in query_str),
            'has_or_equals': int('OR' in query_str and '=' in query_str),
            'has_sleep': int('SLEEP' in query_str or 'WAITFOR' in query_str),
            'has_comments': int('--' in query_str or '/*' in query_str or '#' in query_str),
            'url_encoded': int('%' in query_str),
            'has_info_schema': int('INFORMATION_SCHEMA' in query_str),
            'has_exec': int('EXEC' in query_str or 'EXECUTE' in query_str),
        }
        
        features = pd.concat([features, pd.DataFrame([feature_dict])], ignore_index=True)
    
    return features

print("Extracting features...")
print("This may take a few minutes for large datasets...\n")

# Extract numeric features
print("1. Extracting numeric features...")
numeric_features_train = extract_numeric_features(X_train)
numeric_features_val = extract_numeric_features(X_val)
numeric_features_test = extract_numeric_features(X_test)
print(f"   ✓ Numeric features shape: {numeric_features_train.shape}")

# Fit TF-IDF vectorizer on training data
print("\n2. Fitting TF-IDF vectorizer...")
tfidf = TfidfVectorizer(**CONFIG['tfidf'])
tfidf_train = tfidf.fit_transform(X_train)
tfidf_val = tfidf.transform(X_val)
tfidf_test = tfidf.transform(X_test)
print(f"   ✓ TF-IDF features shape: {tfidf_train.shape}")
print(f"   ✓ Vocabulary size: {len(tfidf.vocabulary_):,}")

# Scale numeric features
print("\n3. Scaling numeric features...")
scaler = StandardScaler()
numeric_scaled_train = scaler.fit_transform(numeric_features_train)
numeric_scaled_val = scaler.transform(numeric_features_val)
numeric_scaled_test = scaler.transform(numeric_features_test)

# Combine TF-IDF and numeric features
from scipy.sparse import hstack, csr_matrix

print("\n4. Combining features...")
X_train_full = hstack([tfidf_train, csr_matrix(numeric_scaled_train)])
X_val_full = hstack([tfidf_val, csr_matrix(numeric_scaled_val)])
X_test_full = hstack([tfidf_test, csr_matrix(numeric_scaled_test)])

print(f"\n✓ Final feature matrices:")
print(f"  - Train: {X_train_full.shape}")
print(f"  - Val:   {X_val_full.shape}")
print(f"  - Test:  {X_test_full.shape}")

# Save feature extractors
print("\n5. Saving feature extractors...")
joblib.dump(tfidf, ML_MODELS_DIR / 'tfidf_vectorizer.joblib')
joblib.dump(scaler, ML_MODELS_DIR / 'numeric_scaler.joblib')
print("   ✓ Saved to ml/models/")